In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# write all code in one cell

# ========================Load data=========================
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense,Concatenate
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
import pickle

with open('/content/drive/My Drive/one_word_spell_correction_noblog_dataset','rb') as f:
    train_texts,train_output = pickle.load(f)

# convert string to lower case
# train_texts = train_df[1].values

train_texts = [s.lower() for s in train_texts]
for i in range(len(train_texts)):
#   print(i)
  train_texts[i] = train_texts[i].replace(' ','')

# test_texts = test_df[1].values
# test_texts = [s.lower() for s in test_texts]

# =======================Convert string to index================
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
# tk.fit_on_texts(train_texts)
# If we already have a character list, then replace the tk.word_index
# If not, just skip below part

# -----------------------Skip part start--------------------------
# construct a new vocabulary
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789 "
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
# -----------------------Skip part end----------------------------

# Convert string to index
train_sequences = tk.texts_to_sequences(train_texts)
# my_texts = list(map(sequence_to_text, sentences))

# my_texts = sequence_to_text(train_sequences[:2])
# print(type(train_sequences))
# test_texts = tk.texts_to_sequences(test_texts)

# Creating texts 
# Padding
train_data = pad_sequences(train_sequences, maxlen=13, padding='post')
# print(type(train_data))
# test_data = pad_sequences(test_texts, maxlen=50, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
# print(my_texts)
# train_output = np.array(train_output, dtype='float32')

# =======================Get classes================
# train_classes = train_df[0].values
# train_class_list = [x - 1 for x in train_output]
# train_class_list = train_output
# test_classes = test_df[0].values
# test_class_list = [x - 1 for x in test_classes]

# from keras.utils import to_categorical
from numpy import argmax
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(train_output)
# inverted = label_encoder.inverse_transform([train_output[0]])
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[-1, :])])
print(inverted)
# enc = OneHotEncoder()#handle_unknown='ignore')
# enc.fit(train_output)
# train_classes = to_categorical(train_output)
# test_classes = to_categorical(test_class_list)




In [0]:
# =====================Char CNN=======================
# parameter
from keras.layers import Concatenate,Add,BatchNormalization,Dropout,Bidirectional,LSTM
from keras.utils import plot_model

input_size = 13
vocab_size = len(tk.word_index)
embedding_size = len(tk.word_index)
conv_layers = [[256, 10],
               [256, 10],
               [256, 7],
               [256, 7],
               [256, 7],
               [256, 3],
               [256, 3],]
fully_connected_layers = [128, 128]
num_of_classes = len(np.unique(integer_encoded))
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

# Embedding weights
embedding_weights = []  # (70, 69)
embedding_weights.append(np.zeros(vocab_size))  # (0, 69)

for char, i in tk.word_index.items():  # from index 1 to 69
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
print('Load')

# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights],
                            trainable=False)

# Model Construction
# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding
x = embedding_layer(inputs)
# Conv
# for filter_num, filter_size in conv_layers:
#     if filter_size == 10:
# #       if k == 0:
#         x_10 = Conv1D(filter_num, filter_size)(x)
#         x_10 = Activation('relu')(x_10)
# #         k = 1
# #       else:
# #         x_10 = Conv1D(filter_num, filter_size)(x_10)
# #         x_10 = Activation('relu')(x_10)
#     elif filter_size == 7:
# #       if d == 0:
#         x_7 = Conv1D(filter_num, filter_size)(x)
#         x_7 = Activation('relu')(x_7)
# #         d = 1
# #       else:
# #         x_7 = Conv1D(filter_num, filter_size)(x_7)
# #         x_7 = Activation('relu')(x_7)
#     elif filter_size == 5:
# #       if v == 0:
#         x_5 = Conv1D(filter_num, filter_size)(x)
#         x_5 = Activation('relu')(x_5)
# #         v = 1
# #       else:
# #         x_5 = Conv1D(filter_num, filter_size)(x_5)
# #         x_5 = Activation('relu')(x_5)
#     elif filter_size == 3:
# #       if b == 0:
#         x_3 = Conv1D(filter_num, filter_size)(x)
#         x_3 = Activation('relu')(x_3)
# #         b = 1
# #       else:
# #         x_3 = Conv1D(filter_num, filter_size)(x)
# #         x_3 = Activation('relu')(x_3)
        
# x = Add()([x_10,x_7,x_5,x_3])
# conv_layers = [[256,10],[256,10],[256,7],[256,7],[256,7],[256,3],[256,3]]
conv = Conv1D(256, 13,padding='same')(x)
conv = BatchNormalization()(conv)
conv = Activation('elu')(conv)
conv = Dropout(0.3)(conv)
conv = Conv1D(100, 13,padding='same')(conv)
conv = BatchNormalization()(conv)
conv = Activation('elu')(conv)
conv = Dropout(0.3)(conv)

x = Bidirectional(LSTM(25,return_sequences=True))(x)

# for filter_num, filter_size in conv_layers:
# #     if filter_size == 10:
#       x = Conv1D(filter_num, filter_size,padding='same')(x)
#       x = BatchNormalization()(x)
#       x = Activation('relu')(x)
#       x = Dropout(0.3)(x)
#       x = Activation('elu')(x)

#     if pooling_size != -1:
#         x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Concatenate()([conv,x])

# bi = Flatten()(x)
x = Flatten()(x)  # (None, 8704)
# Fully connected layers
# for dense_size in fully_connected_layers:
#     x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
#     x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  # Adam, categorical_crossentropy
model.summary()

# # 1000 training samples and 100 testing samples
# indices = np.arange(train_data.shape[0])
# np.random.shuffle(indices)
#
# x_train = train_data[indices][:1000]
# y_train = train_classes[indices][:1000]
#
# x_test = test_data[:100]
# y_test = test_classes[:100]

indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_classes[indices]

plot_model(model, to_file='/content/drive/My Drive/singewordvisualmodel.png')

# Training


In [0]:
model.fit(x_train, y_train,
          validation_split=0.01,
          batch_size=32,
          epochs=20,shuffle=True)

In [0]:
def data(x):
  # Convert string to index
#   train_sequences = tk.texts_to_sequences(train_texts)
  test_texts = tk.texts_to_sequences(x)

  # Padding
#   train_data = pad_sequences(train_sequences, maxlen=50, padding='post')
  test_data = pad_sequences(test_texts, maxlen=13, padding='post')

  # Convert to numpy array
  x = np.array(test_data, dtype='float32')
  return x

In [0]:
predicted = model.predict(data(['wedd']))
# print(predicted)
# print(predicted[-1,:])
inverted = label_encoder.inverse_transform([argmax(predicted[-1, :])])
print(inverted)


In [0]:
try:
    model_json = model.to_json()
    with open("/content/drive/My Drive/final_spell_correction/cnn_bilstm_one_word_spell_correction.json", "w") as json_file:
        json_file.write(model_json)
except:
    pass
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/final_spell_correction/cnn_bilstm_one_word_correction.h5")
print("Saved model to disk")

In [0]:
# two word spell correction

In [0]:
# write all code in one cell

# ========================Load data=========================
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense,Concatenate
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
import pickle

with open('/content/drive/My Drive/two_word_spell_correction_noblog_dataset','rb') as f:
    train_texts,train_output = pickle.load(f)
train_output_1 = []
for k,i in enumerate(train_output):
    train_output_1.append(i[1])
    del train_output[k][1]
# convert string to lower case
# train_texts = train_df[1].values
# for l,i in enumerate(train_texts):
#   if isinstance(i,list):
#     train_texts[l] = i[0]
# train_texts = [s.lower() for s in train_texts]
# for i in range(len(train_texts)):
# #   print(i)
#   train_texts[i] = train_texts[i].replace(' ','')

# test_texts = test_df[1].values
# test_texts = [s.lower() for s in test_texts]

# =======================Convert string to index================
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)
# If we already have a character list, then replace the tk.word_index
# If not, just skip below part

# -----------------------Skip part start--------------------------
# construct a new vocabulary
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789 "
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
# -----------------------Skip part end----------------------------

# Convert string to index
train_sequences = tk.texts_to_sequences(train_texts)
# test_texts = tk.texts_to_sequences(test_texts)

# Padding
train_data = pad_sequences(train_sequences, maxlen=13, padding='post')
# test_data = pad_sequences(test_texts, maxlen=50, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
# train_output = np.array(train_output, dtype='float32')

# =======================Get classes================
# train_classes = train_df[0].values
# train_class_list = [x - 1 for x in train_output]
# train_class_list = train_output
# test_classes = test_df[0].values
# test_class_list = [x - 1 for x in test_classes]

# from keras.utils import to_categorical
from numpy import argmax
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(train_output)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[-1, :])])
print(inverted)

label_encoder_1 = LabelEncoder()
integer_encoded_1 = label_encoder_1.fit_transform(train_output_1)
print(integer_encoded_1)
# binary encode
onehot_encoder_1 = OneHotEncoder(sparse=False)
integer_encoded_1 = integer_encoded_1.reshape(len(integer_encoded_1), 1)
onehot_encoded_1 = onehot_encoder_1.fit_transform(integer_encoded_1)
print(onehot_encoded_1)
# invert first example
inverted_1 = label_encoder_1.inverse_transform([argmax(onehot_encoded_1[-1, :])])
print(inverted_1)

# label_encoder_2 = LabelEncoder()
# integer_encoded_2 = label_encoder_2.fit_transform(train_output_2)
# print(integer_encoded_2)
# # binary encode
# onehot_encoder_2 = OneHotEncoder(sparse=False)
# integer_encoded_2 = integer_encoded_2.reshape(len(integer_encoded_2), 1)
# onehot_encoded_2 = onehot_encoder_2.fit_transform(integer_encoded_2)
# print(onehot_encoded_2)
# # invert first example
# inverted_2 = label_encoder_2.inverse_transform([argmax(onehot_encoded_2[-1, :])])
# print(inverted_2)


# enc = OneHotEncoder()#handle_unknown='ignore')
# enc.fit(train_output)
# train_classes = to_categorical(train_output)
# test_classes = to_categorical(test_class_list)




In [0]:
train_classes = onehot_encoded
train_classes_1 = onehot_encoded_1

In [0]:
# =====================Char CNN=======================
# parameter
from keras.layers import Concatenate,Add,BatchNormalization,Dropout,Bidirectional,LSTM
from keras.utils import plot_model

input_size = 13
vocab_size = len(tk.word_index)
embedding_size = len(tk.word_index)
conv_layers = [[256, 10],
               [256, 10],
               [256, 7],
               [256, 7],
               [256, 7],
               [256, 3],
               [256, 3],]
fully_connected_layers = [128, 128]
num_of_classes = len(np.unique(integer_encoded))
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

# Embedding weights
embedding_weights = []  # (70, 69)
embedding_weights.append(np.zeros(vocab_size))  # (0, 69)

for char, i in tk.word_index.items():  # from index 1 to 69
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
print('Load')

# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights],
                            trainable=False)

# Model Construction
# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding
x = embedding_layer(inputs)
# Conv
# for filter_num, filter_size in conv_layers:
#     if filter_size == 10:
# #       if k == 0:
#         x_10 = Conv1D(filter_num, filter_size)(x)
#         x_10 = Activation('relu')(x_10)
# #         k = 1
# #       else:
# #         x_10 = Conv1D(filter_num, filter_size)(x_10)
# #         x_10 = Activation('relu')(x_10)
#     elif filter_size == 7:
# #       if d == 0:
#         x_7 = Conv1D(filter_num, filter_size)(x)
#         x_7 = Activation('relu')(x_7)
# #         d = 1
# #       else:
# #         x_7 = Conv1D(filter_num, filter_size)(x_7)
# #         x_7 = Activation('relu')(x_7)
#     elif filter_size == 5:
# #       if v == 0:
#         x_5 = Conv1D(filter_num, filter_size)(x)
#         x_5 = Activation('relu')(x_5)
# #         v = 1
# #       else:
# #         x_5 = Conv1D(filter_num, filter_size)(x_5)
# #         x_5 = Activation('relu')(x_5)
#     elif filter_size == 3:
# #       if b == 0:
#         x_3 = Conv1D(filter_num, filter_size)(x)
#         x_3 = Activation('relu')(x_3)
# #         b = 1
# #       else:
# #         x_3 = Conv1D(filter_num, filter_size)(x)
# #         x_3 = Activation('relu')(x_3)
        
# x = Add()([x_10,x_7,x_5,x_3])
# conv_layers = [[256,10],[256,10],[256,7],[256,7],[256,7],[256,3],[256,3]]
conv = Conv1D(256, 13,padding='same')(x)
conv = BatchNormalization()(conv)
conv = Activation('elu')(conv)
conv = Dropout(0.3)(conv)
conv = Conv1D(100, 13,padding='same')(conv)
conv = BatchNormalization()(conv)
conv = Activation('elu')(conv)
conv = Dropout(0.3)(conv)

x = Bidirectional(LSTM(25,return_sequences=True))(x)

# for filter_num, filter_size in conv_layers:
# #     if filter_size == 10:
#       x = Conv1D(filter_num, filter_size,padding='same')(x)
#       x = BatchNormalization()(x)
#       x = Activation('relu')(x)
#       x = Dropout(0.3)(x)
#       x = Activation('elu')(x)

#     if pooling_size != -1:
#         x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Concatenate()([conv,x])

# bi = Flatten()(x)
x = Flatten()(x)  # (None, 8704)
# Fully connected layers
# for dense_size in fully_connected_layers:
#     x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
#     x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
predictions_1 = Dense(num_of_classes+1, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=[predictions,predictions_1])
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  # Adam, categorical_crossentropy
model.summary()

# # 1000 training samples and 100 testing samples
# indices = np.arange(train_data.shape[0])
# np.random.shuffle(indices)
#
# x_train = train_data[indices][:1000]
# y_train = train_classes[indices][:1000]
#
# x_test = test_data[:100]
# y_test = test_classes[:100]

indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_classes[indices]
y_train_1 = train_classes_1[indices]

# plot_model(model, to_file='/content/drive/My Drive/singewordvisualmodel.png')
# 
# Training


In [0]:
model.fit(x_train, [y_train,y_train_1],
          validation_split=0.01,
          batch_size=32,
          epochs=10,shuffle=True)

In [0]:
def data(x):
  # Convert string to index
#   train_sequences = tk.texts_to_sequences(train_texts)
  test_texts = tk.texts_to_sequences(x)

  # Padding
#   train_data = pad_sequences(train_sequences, maxlen=50, padding='post')
  test_data = pad_sequences(test_texts, maxlen=13, padding='post')

  # Convert to numpy array
  x = np.array(test_data, dtype='float32')
  return x
predicted,predicted_1 = model.predict(data(['']))
# print(predicted)
# print(predicted[-1,:])
inverted = label_encoder.inverse_transform([argmax(predicted[-1, :])])
inverted_1 = label_encoder_1.inverse_transform([argmax(predicted_1[-1, :])])
# inverted_2 = label_encoder_2.inverse_transform([argmax(predicted_2[-1, :])])


print(inverted,inverted_1)


In [0]:
try:
    model_json = model.to_json()
    with open("/content/drive/My Drive/final_spell_correction/cnn_bilstm_two_word_classification.json", "w") as json_file:
        json_file.write(model_json)
except:
    pass
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/final_spell_correction/cnn_bilstm_two_word_classification_classification.h5")
print("Saved model to disk")